In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import squidpy as sq
from matplotlib import rcParams

FIGSIZE = (3, 3)
rcParams["figure.figsize"] = FIGSIZE

In [ ]:
path = "../results/RU151.h5ad"
adata = sc.read_h5ad(path)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 3))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)

axs[2].set_title("Nucleus area")
sns.histplot(
    adata.obs["nucleus_area"],
    kde=False,
    ax=axs[2],
)


In [ ]:
sc.pl.umap(
    adata,
    color=[
        "total_counts",
        "n_genes_by_counts",
        "leiden",
    ],
    wspace=0.4,
)

In [ ]:
sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color=[
        "leiden",
    ],
    wspace=0.4,
)

In [ ]:
n_genes = 10
sc.pl.rank_genes_groups(adata, n_genes = n_genes)

In [ ]:
gene_info = pd.read_csv("/data/chanjlab/Xenium.lung.NE_plasticity.010124/ref/Final_Gene_Panel_Design.081523/Xenium_hLung_v1_metadata.csv")
gene_info_2 = pd.read_table("/data/chanjlab/Xenium.lung.NE_plasticity.010124/ref/Final_Gene_Panel_Design.081523/Xenium.custom_gene_panel.v5.txt")
genes = pd.concat([gene_info.Gene, gene_info_2.Gene])
annotations = pd.concat([gene_info.Annotation, gene_info_2.Info])
gene_annot_map = dict(zip(genes, annotations))

# Get ranked genes from adata
ranked_genes = adata.uns['rank_genes_groups']
gene_names = ranked_genes['names']
scores = ranked_genes['scores']

genes_dict = {}
for group in gene_names.dtype.names:
    genes_with_score = list(zip(gene_names[group], scores[group]))
    sorted_genes = sorted(genes_with_score, key=lambda x: x[1], reverse=True)  # Sort by fold change in descending order
    top_genes = [gene for gene, fold_change in sorted_genes[:n_genes]]  # Take the top n_genes
    genes_dict[group] = top_genes

# Get the result as a list of dictionaries
def find_keys_for_all_values(gene_annot_map, genes_dict):
    result = []
    for group, genes in genes_dict.items():
        for gene in genes:
            annotation = gene_annot_map.get(gene, None)
            result.append({'group': group, 'gene': gene, 'annotation': annotation})
    return result
table_data = find_keys_for_all_values(gene_annot_map, genes_dict)

# Convert to pandas DataFrame
df = pd.DataFrame(table_data).drop_duplicates()
pd.set_option('display.max_rows', 500)
df